In [185]:
import string

import pandas as pd
import numpy as np

import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [186]:

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rounak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [190]:
data = pd.read_csv(r'C:\Users\Rounak\OneDrive\Desktop\Machine Learning\SMS_Spam_Detection\spam.csv', encoding='latin-1')


In [ ]:
def clean_text(text):
    if pd.isna(text):
        return "emptytext"
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    words = text.split()  # Simple tokenizer instead of word_tokenize
    stop_words = set([
        'i','me','my','myself','we','our','ours','ourselves','you','your',
        'yours','yourself','yourselves','he','him','his','himself','she','her',
        'hers','herself','it','its','itself','they','them','their','theirs',
        'themselves','what','which','who','whom','this','that','these','those',
        'am','is','are','was','were','be','been','being','have','has','had',
        'having','do','does','did','doing','a','an','the','and','but','if','or',
        'because','as','until','while','of','at','by','for','with','about',
        'against','between','into','through','during','before','after','above',
        'below','to','from','up','down','in','out','on','off','over','under',
        'again','further','then','once','here','there','when','where','why',
        'how','all','any','both','each','few','more','most','other','some',
        'such','no','nor','not','only','own','same','so','than','too','very',
        's','t','can','will','just','don','should','now'
    ])
    words = [word for word in words if word not in stop_words]
    return ' '.join(words) if words else "emptytext"


In [ ]:
data['cleaned'] = data['sms'].apply(clean_text)

In [ ]:
texts = data['cleaned']
labels = data['label']
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
# Make sure train_texts and test_texts contain valid strings
train_texts = ["emptytext" if pd.isna(text) or text.strip() == "" else text for text in train_texts]
test_texts = ["emptytext" if pd.isna(text) or text.strip() == "" else text for text in test_texts]




In [ ]:
vectorizer = TfidfVectorizer(max_features=2500)
train_vectors = vectorizer.fit_transform(train_texts)
test_vectors = vectorizer.transform(test_texts)


In [ ]:
classifier = MultinomialNB()
classifier.fit(train_vectors, train_labels)
preds = classifier.predict(test_vectors)

In [ ]:
score = accuracy_score(test_labels, preds)
print('Model accuracy:', score)
print('Breakdown')
print(classification_report(test_labels, preds, target_names=['ham', 'spam']))

Model accuracy: 0.9757847533632287
Breakdown
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       965
        spam       1.00      0.82      0.90       150

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.94      1115
weighted avg       0.98      0.98      0.97      1115



In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(classifier, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [ ]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
new_sms = ['free gift click now']
new_sms = new_sms[0].lower()
new_sms = re.sub(r'[^a-z0-9\s]', '', new_sms)
words = new_sms.split()
words = [word for word in words if word not in ENGLISH_STOP_WORDS]
clean_sms = ' '.join(words)
print(clean_sms)


free gift click


In [ ]:
with open('model.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [ ]:
sms_vector = vectorizer.transform([clean_sms])
result = classifier.predict(sms_vector)[0]
if result == 1:
    print('This is a Spam')
else:
    print('This is Not Spam')

This is a Spam
